In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [22]:
#test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [23]:
#print('data({0[0]},{0[1]})'.format(data.shape))
print (train.head(5))

   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0         

In [24]:
train_x = train.iloc[:,1:].values
train_x = train_x.astype(np.float)

# convert from [0:255] => [0.0:1.0]
train_x= np.multiply(train_x, 1.0 / 255.0)



train_y = train.iloc[:,0].values

In [25]:
train_x.shape

(42000, 784)

In [26]:
len(train_y)

42000

In [27]:
batch_size = 128
num_classes = 10
epochs = 30

# input image dimensions
img_rows, img_cols = 28, 28


In [28]:
from __future__ import print_function
import keras
#from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 

In [29]:
train_y = keras.utils.to_categorical(train_y, num_classes)
#validation_labels = keras.utils.to_categorical(validation_labels, num_classes)


In [30]:
train_y.shape 


(42000, 10)

In [31]:
train_x = train_x.reshape(-1,28,28,1)

In [32]:
x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, test_size = 0.1, random_state=2)

In [33]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_val =x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


In [35]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))


In [36]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)         

In [37]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [47]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


#datagen.fit(X_train)
train_gen = datagen.flow(x_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(x_val, y_val, batch_size=batch_size)

In [ ]:
history=model.fit_generator(train_gen, 
                              epochs =epochs, 
                              steps_per_epoch = x_train.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = x_val.shape[0] // batch_size)

Epoch 1/30
295/295 [==============================] - 144s 487ms/step - loss: 0.1011 - accuracy: 0.9702 - val_loss: 0.0074 - val_accuracy: 0.9832
Epoch 2/30
295/295 [==============================] - 144s 487ms/step - loss: 0.0636 - accuracy: 0.9815 - val_loss: 0.0410 - val_accuracy: 0.9870
Epoch 3/30
295/295 [==============================] - 142s 481ms/step - loss: 0.0502 - accuracy: 0.9850 - val_loss: 0.0243 - val_accuracy: 0.9892
Epoch 4/30
295/295 [==============================] - 143s 486ms/step - loss: 0.0465 - accuracy: 0.9851 - val_loss: 0.0336 - val_accuracy: 0.9899
Epoch 5/30
295/295 [==============================] - 143s 484ms/step - loss: 0.0438 - accuracy: 0.9866 - val_loss: 0.0101 - val_accuracy: 0.9899
Epoch 6/30
 52/295 [====>.........................] - ETA: 1:56 - loss: 0.0376 - accuracy: 0.9892

In [41]:
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [42]:
test =test .iloc[:,:].values
test = test .reshape(-1,28,28,1)

results = model.predict(test)
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")


In [43]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)